<a href="https://colab.research.google.com/github/adsferreira/yolov3_cattle_detector/blob/master/Train_YoloV3_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Connect google drive**

In [3]:
# Check if NVIDIA GPU is enabled
!nvidia-smi

Tue Aug 11 18:18:23 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.57       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
 15_04_17_50:5:50.mat
'2019_Formulario de Solicitacao de Componentes Curriculares para Atribuicao de Aulas.xlsx'
 33.JPG
 Adriano_041664_Qualificacao.docx.gdoc
'AdrianoDaSilvaFerreira_documentosComprobatorios (1).zip'
 AdrianoDaSilvaFerreira_documentosComprobatorios.zip
 AdrianoFerreira.cpp
 analisis-arquivodxf.xls
'Android 6.pdf.zip'
'anexo_ii_formulario_cap_externa (1).gdoc'
 anexo_ii_formulario_cap_externa.gdoc
'anexo_iii_dispensa_de_atividades_cap_externa_ifsp_agosto_2013 (1).gdoc'
 anexo_iii_dispensa_de_atividades_cap_externa_ifsp_agosto_2013.gdoc
'Anexos resolucao 109 (1).gsheet'
'Anexos resolucao 109 (2).gsheet'
'Anexos resolucao 109.gsheet'
'Anexos resolucao 109.xls'
'anexo vi.formulrio_afast_pas ifsp 2013.gdoc'
'Artificial Neural Network'
'Artigo Rafa.gdoc'
'Artigo Rafa.odt'
 ASOC17_files
 Ata.gdoc
'Avaliação Diagnóstica - DSWI6 - 1s2019.gform'


**1) Clone the Darknet**



In [5]:
!git clone https://github.com/AlexeyAB/darknet

Cloning into 'darknet'...
remote: Enumerating objects: 43, done.
remote: Counting objects: 100% (43/43), done.
remote: Compressing objects: 100% (34/34), done.
remote: Total 14240 (delta 21), reused 22 (delta 8), pack-reused 14197
Receiving objects: 100% (14240/14240), 12.77 MiB | 16.56 MiB/s, done.
Resolving deltas: 100% (9673/9673), done.


**2) Compile Darknet using Nvidia GPU**


In [6]:
# change makefile to have GPU and OPENCV enabled
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!make

/content/darknet
mkdir -p ./obj/
mkdir -p backup
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:926:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
                 float rgb[3];
                       ^~~
./src/image_opencv.cpp: In function ‘void draw_train_loss(char*, void**, int, float, float, int, int, float, int, char*, float, int, int, double)’:
./src/image_opencv.cpp:1127:13: warning: this ‘if’ clause does not guard... [-Wmisleading-indentation]
             if (iteration_old == 0)
           

**3) Configure Darknet network for training YOLO V3**

In [7]:
!cp cfg/yolov3.cfg cfg/yolov3_training.cfg

In [8]:
!sed -i 's/batch=1/batch=64/' cfg/yolov3_training.cfg
!sed -i 's/subdivisions=1/subdivisions=16/' cfg/yolov3_training.cfg
!sed -i 's/max_batches = 500200/max_batches = 4000/' cfg/yolov3_training.cfg
!sed -i '610 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '696 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '783 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '603 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '689 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '776 s@filters=255@filters=18@' cfg/yolov3_training.cfg

In [9]:
# Create folder on google drive so that we can save there the weights
!mkdir "/mydrive/yolov3"

mkdir: cannot create directory ‘/mydrive/yolov3’: File exists


In [10]:
!echo "Koala" > data/obj.names
!echo -e 'classes= 1\ntrain  = data/train.txt\nvalid  = data/test.txt\nnames = data/obj.names\nbackup = /mydrive/yolov3' > data/obj.data
!mkdir data/obj

In [11]:
# Download weights darknet model 53
!wget https://pjreddie.com/media/files/darknet53.conv.74

--2020-08-11 18:20:16--  https://pjreddie.com/media/files/darknet53.conv.74
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 162482580 (155M) [application/octet-stream]
Saving to: ‘darknet53.conv.74’

darknet53.conv.74   100%[===================>] 154.96M   315KB/s    in 8m 17s  

2020-08-11 18:28:35 (319 KB/s) - ‘darknet53.conv.74’ saved [162482580/162482580]



**4) Extract Images**

The images need to be inside a zip archive called "images.zip" and they need to be inside the folder "yolov3" on Google Drive

In [12]:
!unzip /mydrive/yolov3/images.zip -d data/obj

Archive:  /mydrive/yolov3/images.zip
  inflating: data/obj/0a3cb61a6688da2e.jpg  
  inflating: data/obj/0a3cb61a6688da2e.txt  
  inflating: data/obj/0a5c8e8d9ef62186.jpg  
  inflating: data/obj/0a5c8e8d9ef62186.txt  
  inflating: data/obj/0a6fc72e2e31018c.jpg  
  inflating: data/obj/0a6fc72e2e31018c.txt  
  inflating: data/obj/0a12aad70c5c6310.jpg  
  inflating: data/obj/0a12aad70c5c6310.txt  
  inflating: data/obj/0a036c2bd1b5a4d4.jpg  
  inflating: data/obj/0a036c2bd1b5a4d4.txt  
  inflating: data/obj/0a59fef54530bb19.jpg  
  inflating: data/obj/0a59fef54530bb19.txt  
  inflating: data/obj/0b97e7818af0537d.jpg  
  inflating: data/obj/0b97e7818af0537d.txt  
  inflating: data/obj/0b2788dbb074b2ca.jpg  
  inflating: data/obj/0b2788dbb074b2ca.txt  
  inflating: data/obj/0b4783fe4e363741.jpg  
  inflating: data/obj/0b4783fe4e363741.txt  
  inflating: data/obj/0bb22bf977396515.jpg  
  inflating: data/obj/0bb22bf977396515.txt  
  inflating: data/obj/0befe1719a23c177.jpg  
  inflating: data/

In [13]:
# We're going to convert the class index on the .txt files. As we're working with only one class, it's supposed to be class 0.
# If the index is different from 0 then we're going to change it.
import glob
import os
import re

txt_file_paths = glob.glob(r"data/obj/*.txt")
for i, file_path in enumerate(txt_file_paths):
    # get image size
    with open(file_path, "r") as f_o:
        lines = f_o.readlines()

        text_converted = []
        for line in lines:
            print(line)
            numbers = re.findall("[0-9.]+", line)
            print(numbers)
            if numbers:

                # Define coordinates
                text = "{} {} {} {} {}".format(0, numbers[1], numbers[2], numbers[3], numbers[4])
                text_converted.append(text)
                print(i, file_path)
                print(text)
        # Write file
        with open(file_path, 'w') as fp:
            for item in text_converted:
                fp.writelines("%s\n" % item)

Streaming output truncated to the last 5000 lines.

['0', '0.414296875', '0.458125333333', '0.1525', '0.325']
711 data/obj/84b64ccb88371d98.txt
0 0.414296875 0.458125333333 0.1525 0.325
0 0.42328125 0.697239583333 0.38375 0.301667

['0', '0.42328125', '0.697239583333', '0.38375', '0.301667']
711 data/obj/84b64ccb88371d98.txt
0 0.42328125 0.697239583333 0.38375 0.301667
0 0.6416796875 0.602343416667 0.225625 0.513334

['0', '0.6416796875', '0.602343416667', '0.225625', '0.513334']
711 data/obj/84b64ccb88371d98.txt
0 0.6416796875 0.602343416667 0.225625 0.513334
0 0.823125 0.518749666667 0.15625 0.146667

['0', '0.823125', '0.518749666667', '0.15625', '0.146667']
711 data/obj/84b64ccb88371d98.txt
0 0.823125 0.518749666667 0.15625 0.146667
0 0.4648828125 0.455989583333 0.10375 0.113333

['0', '0.4648828125', '0.455989583333', '0.10375', '0.113333']
712 data/obj/aed6213c0fb64da4.txt
0 0.4648828125 0.455989583333 0.10375 0.113333
0 0.2693359375 0.834583 0.139375 0.189167

['0', '0.269335937

In [14]:
import glob
images_list = glob.glob("data/obj/*.jpg")
print(images_list)

['data/obj/329f50080f0d8147.jpg', 'data/obj/6c66e8dba1073703.jpg', 'data/obj/1b2686f2fe844fce.jpg', 'data/obj/5995ecafa740fe40.jpg', 'data/obj/844ddba8f9d46f62.jpg', 'data/obj/deadd87e3e9f59e5.jpg', 'data/obj/6dd1d0e95042d0ee.jpg', 'data/obj/58a8f279894cd742.jpg', 'data/obj/4d6071d0ddec965c.jpg', 'data/obj/0a12aad70c5c6310.jpg', 'data/obj/6a4ab1306df6261a.jpg', 'data/obj/a18e1d850b3a09ab.jpg', 'data/obj/6a996a74d6d5f5b7.jpg', 'data/obj/042a28c8055ea2a2.jpg', 'data/obj/ec217c7de1a6f7f5.jpg', 'data/obj/c2bbf24b0ece1696.jpg', 'data/obj/d3a0376983e36440.jpg', 'data/obj/2c36f984d517e4c2.jpg', 'data/obj/54714b797c828a2a.jpg', 'data/obj/350c6ea4fd80169d.jpg', 'data/obj/ee93f04fa6227c1a.jpg', 'data/obj/1700ed1ce0bfbf1e.jpg', 'data/obj/46b5ec0aec0c3627.jpg', 'data/obj/0055d46c79cb30d2.jpg', 'data/obj/24eee01ea4013d6f.jpg', 'data/obj/f9081c9c25dedc88.jpg', 'data/obj/57388b0d2a736382.jpg', 'data/obj/063c7a39884d83a8.jpg', 'data/obj/72fdc9962ac875b4.jpg', 'data/obj/2a470da9213eeba0.jpg', 'data/obj

In [15]:
#Create training.txt file
file = open("data/train.txt", "w") 
file.write("\n".join(images_list)) 
file.close() 

**6) Start the training**

In [16]:
# Start the training
!./darknet detector train data/obj.data cfg/yolov3_training.cfg darknet53.conv.74 -dont_show

Streaming output truncated to the last 5000 lines.

 3926: 0.735449, 0.793341 avg loss, 0.001000 rate, 5.650063 seconds, 251264 images, 0.163722 hours left
Loaded: 0.000045 seconds
v3 (mse loss, Normalizer: (iou: 0.75, cls: 1.00) Region 82 Avg (IOU: 0.873106, GIOU: 0.868249), Class: 0.999768, Obj: 0.966594, No Obj: 0.001533, .5R: 1.000000, .75R: 1.000000, count: 2, class_loss = 0.001199, iou_loss = 0.023864, total_loss = 0.025064 
v3 (mse loss, Normalizer: (iou: 0.75, cls: 1.00) Region 94 Avg (IOU: 0.745628, GIOU: 0.732222), Class: 0.999761, Obj: 0.717744, No Obj: 0.001294, .5R: 1.000000, .75R: 0.600000, count: 15, class_loss = 1.286470, iou_loss = 0.982676, total_loss = 2.269145 
v3 (mse loss, Normalizer: (iou: 0.75, cls: 1.00) Region 106 Avg (IOU: 0.793623, GIOU: 0.789019), Class: 0.997819, Obj: 0.482195, No Obj: 0.000145, .5R: 1.000000, .75R: 0.750000, count: 8, class_loss = 0.793803, iou_loss = 0.312018, total_loss = 1.105821 
 total_bbox = 905034, rewritten_bbox = 0.789031 % 
v3 (